In [ ]:
"""
LINCS 2020数据加载和预处理脚本 - 内存优化版（修正版）
根据实际数据格式修正字段名
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
import gzip
import shutil
import glob
from scipy.spatial.distance import cosine, pdist, squareform
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional
import warnings
import gc
warnings.filterwarnings('ignore')


class LINCS2020DataLoader:
    """
    加载和预处理LINCS 2020数据 - 内存优化版（修正版）
    严格按照实际数据格式处理
    """
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.gene_info = None
        self.cell_info = None
        self.compound_info = None
        self.inst_info = None  
        self.signatures = None
        self.decompressed_files = []
        
        print(f"\n{'='*80}")
        print(f"🔬 LINCS 2020 Data Loader Initialized (Memory-Optimized v2)")
        print(f"{'='*80}")
        print(f"Data directory: {self.data_dir}")
        print(f"Dataset: Expanded CMap LINCS Resource 2020")
        print(f"{'='*80}\n")
    
    def load_gene_info(self):
        """
        加载基因信息，获取978个landmark genes
        
        实际字段：
        - gene_id: 基因ID (如: 16, 23, 25)
        - gene_symbol: 基因符号 (如: AARS, ABCF1, ABL1)
        - feature_space: 'landmark' 或 'inferred'
        """
        gene_file = self.data_dir / "geneinfo_beta.txt"
        
        if not gene_file.exists():
            raise FileNotFoundError(
                f"❌ Gene info file not found: {gene_file}\n"
                f"   Please download 'geneinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )
        
        print(f"📖 Loading gene information...")
        print(f"   File: {gene_file.name}")
        
        # 读取基因信息
        gene_info = pd.read_csv(gene_file, sep='\t')
        
        print(f"   ✓ Loaded {len(gene_info):,} genes")
        print(f"   ✓ Columns: {list(gene_info.columns)}")
        
        # 筛选landmark genes (feature_space == 'landmark')
        if 'feature_space' in gene_info.columns:
            landmark_mask = gene_info['feature_space'] == 'landmark'
            landmark_genes = gene_info[landmark_mask].copy()
            print(f"   ✓ Using 'feature_space' column to identify landmarks")
        else:
            raise ValueError(
                f"Cannot identify landmark genes. 'feature_space' column not found.\n"
                f"Available columns: {list(gene_info.columns)}"
            )

        print(f"   ✓ Landmark genes: {len(landmark_genes):,}")
        print(f"   ✓ Expected: 978")

        # === 关键改动：保存“行号”作为列索引，而不是去匹配 GCTX 的 meta ===
        # 注意：这里假设 geneinfo_beta.txt 的行顺序和 GCTX 的列顺序一致
        self.landmark_col_indices = np.where(landmark_mask.values)[0]
        print(f"\n   Landmark column indices (first 10): {self.landmark_col_indices[:10]}")

        # 这些集合以后如果还想做 debug 可以保留
        self.landmark_gene_ids = set(landmark_genes['gene_id'].astype(str).values)
        self.landmark_gene_symbols = set(landmark_genes['gene_symbol'].astype(str).values)

        print(f"   Sample IDs for matching: {list(self.landmark_gene_ids)[:5]}")
        print(f"   Sample symbols for matching: {list(self.landmark_gene_symbols)[:5]}")

        # === 关键改动：gene_info 保存“全表”，而不是只保存 landmark 子集 ===
        self.gene_info = gene_info
        return gene_info

    
    def load_cell_info(self):
        """
        加载细胞系信息
        
        实际字段：
        - cell_iname: 细胞系名称
        - cell_lineage: 细胞谱系
        - primary_disease: 主要疾病
        """
        cell_file = self.data_dir / "cellinfo_beta.txt"
        
        if not cell_file.exists():
            raise FileNotFoundError(
                f"❌ Cell info file not found: {cell_file}\n"
                f"   Please download 'cellinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020"
            )
        
        print(f"\n📖 Loading cell information...")
        print(f"   File: {cell_file.name}")
        
        cell_info = pd.read_csv(cell_file, sep='\t')
        
        print(f"   ✓ Loaded {len(cell_info):,} cell lines")
        print(f"   ✓ Columns: {list(cell_info.columns[:10])}...")
        
        # 确定细胞系ID列
        if 'cell_iname' in cell_info.columns:
            print(f"   ✓ Unique cell lines (cell_iname): {cell_info['cell_iname'].nunique()}")
        
        # 显示示例
        print(f"\n   Sample cell lines:")
        for cell in cell_info['cell_iname'].head(5).values:
            print(f"     - {cell}")
        
        self.cell_info = cell_info
        return cell_info
    
    def load_compound_info(self):
        """
        加载化合物信息
        
        实际字段：
        - pert_id: 扰动ID (BRD-XXXXXXXXX)
        - cmap_name: 化合物名称
        - canonical_smiles: SMILES结构
        - target: 靶点
        - moa: 作用机制
        """
        compound_file = self.data_dir / "compoundinfo_beta.txt"
        
        if not compound_file.exists():
            print(f"⚠️  Compound info file not found: {compound_file}")
            print(f"   This file is optional but recommended.")
            return None
        
        print(f"\n📖 Loading compound information...")
        print(f"   File: {compound_file.name}")
        
        compound_info = pd.read_csv(compound_file, sep='\t')
        
        print(f"   ✓ Loaded {len(compound_info):,} compounds")
        print(f"   ✓ Columns: {list(compound_info.columns)}")
        
        if 'pert_id' in compound_info.columns:
            print(f"   ✓ Unique perturbagens: {compound_info['pert_id'].nunique()}")
        
        # 显示示例
        print(f"\n   Sample compounds:")
        for _, row in compound_info.head(3).iterrows():
            name = row.get('cmap_name', 'Unknown')
            print(f"     - {row['pert_id']}: {name}")
        
        self.compound_info = compound_info
        return compound_info
    
    def load_instance_info(self):
        """
        加载实例（well-level）信息：instinfo_beta.txt

        关键字段（LINCS2020）：
        - sample_id: 与 Level4 GCTX 中 ROW/COL meta 里的 id 对应
        - pert_id:   化合物 ID（BRD-...）
        - cell_id: 细胞系 ID
        - pert_time: 处理时间
        - pert_dose: 剂量
        """
        inst_file = self.data_dir / "instinfo_beta.txt"

        if not inst_file.exists():
            raise FileNotFoundError(
                f"❌ Instance info file not found: {inst_file}\n"
                f"   Please download 'instinfo_beta.txt' from:\n"
                f"   https://clue.io/data/CMap2020#LINCS2020\n"
                f"   It is required to obtain pert_id / cell_iname / dose / time for Level4."
            )

        print(f"\n📖 Loading instance information...")
        print(f"   File: {inst_file.name}")

        inst_info = pd.read_csv(inst_file, sep='\t')

        print(f"   ✓ Loaded {len(inst_info):,} instances")
        print(f"   ✓ Columns: {list(inst_info.columns[:10])}...")

        # ✅ 关键修正：LINCS2020 用的是 sample_id，而不是 inst_id
        if 'inst_id' in inst_info.columns:
            self.instance_join_col = 'inst_id'
        elif 'sample_id' in inst_info.columns:
            self.instance_join_col = 'sample_id'
        else:
            raise ValueError(
                f"Neither 'inst_id' nor 'sample_id' found in instinfo_beta.txt. "
                f"Available columns: {list(inst_info.columns)}"
            )

        print(f"   ✓ Using '{self.instance_join_col}' as join key to GCTX metadata")

        self.inst_info = inst_info
        return inst_info

    def decompress_gctx_file(self, gctx_file):
        """如果GCTX文件被压缩，则解压"""
        gctx_file = Path(gctx_file)
        
        if not gctx_file.exists():
            raise FileNotFoundError(f"GCTX file not found: {gctx_file}")
        
        if str(gctx_file).endswith('.gz'):
            print(f"⚠️  Detected compressed GCTX file: {gctx_file.name}")
            
            decompressed_dir = self.data_dir / "_decompressed"
            decompressed_dir.mkdir(exist_ok=True)
            
            output_path = decompressed_dir / gctx_file.stem
            
            if output_path.exists():
                print(f"✓ Found existing decompressed file: {output_path.name}")
                return str(output_path)
            
            print(f"📦 Decompressing...")
            with gzip.open(gctx_file, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out, length=8*1024*1024)
            
            print(f"✓ Decompressed to: {output_path}")
            return str(output_path)
        
        return str(gctx_file)
    
    def read_gctx(self, gctx_file, use_landmark_only=True):
        """
        读取GCTX文件 - 内存优化版
        
        关键优化：
        1. 只读取landmark基因列
        2. 使用float32
        3. 分块读取
        """
        gctx_file = str(gctx_file)
        print(f"\n📖 Reading GCTX file: {Path(gctx_file).name}")
        
        file_size_gb = Path(gctx_file).stat().st_size / (1024**3)
        print(f"   File size: {file_size_gb:.2f} GB")
        
        if file_size_gb > 50:
            print(f"   ⚠️  Large file detected. Using memory-optimized loading...")
        
        gctx_file = self.decompress_gctx_file(gctx_file)
        
        with h5py.File(gctx_file, 'r') as f:
            print(f"📊 Inspecting HDF5 structure...")
            print(f"   Available keys: {list(f.keys())}")
            
            # 定位数据矩阵
            if '/0/DATA/0/matrix' in f:
                matrix_dataset = f['/0/DATA/0/matrix']
                row_path = '/0/META/ROW'
                col_path = '/0/META/COL'
            elif '/matrix' in f:
                matrix_dataset = f['/matrix']
                row_path = '/row'
                col_path = '/col'
            else:
                raise ValueError(f"Cannot find matrix in GCTX file. Available keys: {list(f.keys())}")
            
            matrix_shape = matrix_dataset.shape
            print(f"   ✓ Matrix shape: {matrix_shape} (samples × genes)")
            
            print(f"\n📋 Loading metadata...")
            
            # 读取样本元数据（ROW）
            sample_meta = {}
            if row_path in f:
                print(f"   Reading sample metadata from: {row_path}")
                print(f"   Available row fields: {list(f[row_path].keys())}")
                
                for key in f[row_path].keys():
                    data = f[f'{row_path}/{key}'][:]
                    if data.dtype.kind in ['S', 'O', 'U']:
                        try:
                            sample_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                        except:
                            sample_meta[key] = data.astype(str)
                    else:
                        sample_meta[key] = data
                
                print(f"   ✓ Loaded {len(sample_meta)} sample metadata fields")
            else:
                raise ValueError(f"Cannot find row metadata at: {row_path}")
            
            # 读取基因元数据（COL）
            gene_meta = {}
            if col_path in f:
                print(f"   Reading gene metadata from: {col_path}")
                print(f"   Available col fields: {list(f[col_path].keys())}")
                
                for key in f[col_path].keys():
                    data = f[f'{col_path}/{key}'][:]
                    if data.dtype.kind in ['S', 'O', 'U']:
                        try:
                            gene_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                        except:
                            gene_meta[key] = data.astype(str)
                    else:
                        gene_meta[key] = data
                
                print(f"   ✓ Loaded {len(gene_meta)} gene metadata fields")
            else:
                raise ValueError(f"Cannot find col metadata at: {col_path}")
            
            sample_df = pd.DataFrame(sample_meta)
            gene_df = pd.DataFrame(gene_meta)

            # === 新增：自动检测 ROW/COL 谁是“样本”、谁是“基因” ===
            if self.gene_info is not None:
                n_features_expected = len(self.gene_info)  # geneinfo_beta.txt 有 12,328 行
                # 如果 ROW 的行数刚好等于 geneinfo 的行数，说明 ROW 是“基因”而不是“样本”，需要调换
                if len(sample_df) == n_features_expected and len(gene_df) != n_features_expected:
                    print(f"\n   ⚠️ Detected that ROW metadata has {len(sample_df)} entries, "
                        f"matching geneinfo ({n_features_expected}). Swapping ROW/COL metadata "
                        f"so that samples correspond to matrix rows.")
                    sample_df, gene_df = gene_df, sample_df

            print(f"\n   Sample metadata columns: {list(sample_df.columns[:10])}...")
            print(f"   Gene metadata columns: {list(gene_df.columns)}")

            
            # 确定landmark基因的列索引
            landmark_col_indices = None
            if use_landmark_only:
                print(f"\n🔬 Filtering to landmark genes...")

                # 确保已经加载 geneinfo，并且已经计算好 landmark_col_indices
                if (self.gene_info is None) or (not hasattr(self, "landmark_col_indices")):
                    print(f"   Loading gene info to get landmark indices...")
                    self.load_gene_info()

                # 安全性检查：geneinfo 行数要和 GCTX 的列数一致
                if len(self.gene_info) != matrix_shape[1]:
                    print(f"   ⚠️  Warning: geneinfo rows ({len(self.gene_info)}) "
                        f"!= GCTX feature count ({matrix_shape[1]}). "
                        f"Please double-check that geneinfo_beta.txt matches this GCTX file.")
                
                landmark_col_indices = np.array(self.landmark_col_indices, dtype=int)
                print(f"   ✓ Using {len(landmark_col_indices)} landmark features "
                    f"out of {matrix_shape[1]} total")
                
            # 🔥 内存优化：分块读取数据
            print(f"\n🎯 Loading data (memory-optimized)...")
            if landmark_col_indices is not None:
                print(f"   Reading {len(landmark_col_indices)} columns out of {matrix_shape[1]}...")
                chunk_size = 610000  # 每次读取61万行
                chunks = []
                
                for start_idx in range(0, matrix_shape[0], chunk_size):
                    end_idx = min(start_idx + chunk_size, matrix_shape[0])
                    print(f"   Loading rows {start_idx:,} to {end_idx:,}... ({end_idx/matrix_shape[0]*100:.1f}%)", end='\r')
                    
                    # 读取指定行和列，转为float32
                    chunk = matrix_dataset[start_idx:end_idx, landmark_col_indices].astype(np.float32)
                    chunks.append(chunk)
                    
                    # 每10个chunk合并一次
                    if len(chunks) >= 10:
                        print(f"\n   Consolidating chunks...")
                        merged = np.vstack(chunks)
                        chunks = [merged]
                        gc.collect()
                
                print(f"\n   Finalizing matrix...")
                matrix = np.vstack(chunks) if len(chunks) > 1 else chunks[0]
                del chunks
                gc.collect()
            else:
                # 全量读取
                print(f"   Reading full matrix...")
                matrix = matrix_dataset[:].astype(np.float32)
            
            print(f"   ✓ Final matrix shape: {matrix.shape}")
            print(f"   ✓ Memory usage: {matrix.nbytes / (1024**3):.2f} GB")
            print(f"   ✓ Data type: {matrix.dtype}")
        
        return matrix, sample_df, gene_df
    
    def load_level4_signatures(self):
        """加载Level 4数据"""
        level4_file = self.data_dir / "level4_beta_trt_cp_n1805898x12328.gctx"
        
        if not level4_file.exists():
            pattern = self.data_dir / "level4_beta_trt_cp*.gctx"
            files = glob.glob(str(pattern))
            
            if not files:
                raise FileNotFoundError(
                    f"❌ Level 4 file not found: {level4_file}\n"
                    f"   Please download from: https://clue.io/data/CMap2020#LINCS2020\n"
                    f"   Expected file: level4_beta_trt_cp_n1805898x12328.gctx (82.94 GB)"
                )
            
            level4_file = files[0]
        
        print(f"\n{'='*80}")
        print(f"📖 Loading Level 4 Signatures")
        print(f"{'='*80}")
        print(f"File: {level4_file.name}")
        
        matrix, sample_meta, gene_meta = self.read_gctx(
            level4_file, 
            use_landmark_only=True
        )

        # === 用 instinfo_beta 补齐 pert_id / cell_iname / dose / time ===

        # 1) 把 GCTX 的 id 重命名为 sample_id（和 instinfo2020 对齐）
        if 'id' in sample_meta.columns:
            # GCTX ROW（或者经过 swap 后的“样本 meta”）里的 id 形如
            #   HOG001_A549_6H_X4_F1B10:P11
            # 与 instinfo_beta.txt 中的 sample_id 一致
            sample_meta = sample_meta.rename(columns={'id': 'sample_id'})
        elif 'sample_id' not in sample_meta.columns:
            raise ValueError(
                "Cannot find 'id' or 'sample_id' column in GCTX ROW metadata; "
                "cannot join with instinfo_beta.txt"
            )

        # 2) 读取 instinfo_beta.txt（会在内部设置 self.instance_join_col）
        if self.inst_info is None:
            self.load_instance_info()

        inst_info = self.inst_info

        # 3) 根据 instinfo 中的 join key 决定 merge 列
        join_col = getattr(self, "instance_join_col", None)
        if join_col is None:
            # 理论上不会走到这里，只是做个保险
            if 'sample_id' in inst_info.columns:
                join_col = 'sample_id'
            else:
                join_col = 'inst_id'

        # 这里 LINCS2020 的情况是：join_col == 'sample_id'
        if join_col not in sample_meta.columns:
            # 如果 join_col 是 'inst_id' 而 sample_meta 没有，就再补一次保险
            if join_col == 'inst_id' and 'sample_id' in sample_meta.columns:
                print("   ⚠️ instance_join_col is 'inst_id' but GCTX meta only has 'sample_id'; "
                    "using 'sample_id' as join key instead.")
                join_col = 'sample_id'
            else:
                raise ValueError(
                    f"Join column '{join_col}' not found in GCTX metadata. "
                    f"Available columns: {list(sample_meta.columns)}"
                )

        print(f"   🔗 Merging GCTX metadata with instinfo_beta on '{join_col}' ...")
        merged_meta = sample_meta.merge(inst_info, on=join_col, how='left')

        if 'pert_id' not in merged_meta.columns:
            raise ValueError(
                "After merging with instinfo_beta, 'pert_id' is still missing.\n"
                "Please check that instinfo_beta.txt matches this Level4 dataset."
            )

        n_missing = merged_meta['pert_id'].isna().sum()
        if n_missing > 0:
            print(f"   ⚠️ instinfo merge: {n_missing:,} rows have missing pert_id")

        self.signatures = {
            'matrix': matrix,
            'row_meta': merged_meta,
            'col_meta': gene_meta
        }

        return matrix, merged_meta, gene_meta

    
    def calculate_cosine_similarity_to_nearest_replicate(
        self, 
        matrix: np.ndarray, 
        pert_ids: pd.Series,
        batch_size: int = 500
    ) -> np.ndarray:
        """
        计算每个样本与其最近replicate的余弦相似度 - 内存优化版
        """
        print(f"\n📊 Calculating cosine similarity to nearest replicate (optimized)...")
        n_samples = len(pert_ids)
        nearest_similarities = np.zeros(n_samples, dtype=np.float32)
        
        unique_perts = pert_ids.unique()
        print(f"   Processing {len(unique_perts):,} compounds...")
        
        from tqdm import tqdm
        
        for pert_id in tqdm(unique_perts, desc="   Computing similarities"):
            pert_mask = pert_ids == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            if len(pert_indices) < 2:
                nearest_similarities[pert_indices] = 0.0
                continue
            
            pert_data = matrix[pert_mask].astype(np.float32)
            n_replicates = len(pert_data)
            
            # 分批计算（大化合物）
            if n_replicates > batch_size:
                max_sims = np.full(n_replicates, -np.inf, dtype=np.float32)
                
                for i in range(0, n_replicates, batch_size):
                    end_i = min(i + batch_size, n_replicates)
                    batch_data = pert_data[i:end_i]
                    
                    sim_batch = cosine_similarity(batch_data, pert_data).astype(np.float32)
                    
                    # 对角线设为-inf
                    for local_idx in range(len(batch_data)):
                        global_idx = i + local_idx
                        sim_batch[local_idx, global_idx] = -np.inf
                    
                    max_sims[i:end_i] = np.max(sim_batch, axis=1)
                    
                    del sim_batch
                    gc.collect()
            else:
                # 小化合物直接计算
                sim_matrix = cosine_similarity(pert_data).astype(np.float32)
                np.fill_diagonal(sim_matrix, -np.inf)
                max_sims = np.max(sim_matrix, axis=1)
                del sim_matrix
            
            nearest_similarities[pert_indices] = max_sims
            del pert_data, max_sims
            gc.collect()
        
        print(f"   ✓ Calculated similarities for {n_samples:,} samples")
        print(f"   Mean similarity: {nearest_similarities.mean():.4f}")
        print(f"   Median similarity: {np.median(nearest_similarities):.4f}")
        
        return nearest_similarities
    
    def prepare_training_data(
        self, 
        min_observations_per_compound=5,
        min_replicate_similarity=0.12,
        dose_range=(1.0, 20.0),
        valid_timepoints=['6 h', '24 h'],
        min_cell_lines=5,
        max_cell_lines=40,
        remove_dos=True
    ):
        """
        准备训练数据 - 内存优化版（修正版）
        
        根据实际数据字段修正处理逻辑
        """
        if self.signatures is None:
            raise ValueError("Please load signatures first using load_level4_signatures()")
        
        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta'].copy()
        col_meta = self.signatures['col_meta']
        
        print(f"\n[DEBUG] row_meta shape after merge: {row_meta.shape}")
        print(f"[DEBUG] row_meta columns (first 15): {list(row_meta.columns[:15])}")
        print(f"[DEBUG] Example row:")
        print(row_meta.iloc[0][['sample_id', 'pert_id', 'pert_type', 'cell_iname', 'pert_time', 'pert_dose']])

        print("\n" + "=" * 80)
        print("🔍 DRUGREFLECTOR QUALITY CONTROL PIPELINE (Memory-Optimized v2)")
        print("=" * 80)
        print(f"Initial samples: {len(row_meta):,}")
        print(f"Initial memory: {matrix.nbytes / (1024**3):.2f} GB")
        
        # 打印可用的元数据列
        print(f"\n📋 Available metadata columns:")
        for col in row_meta.columns[:15]:
            sample_val = row_meta[col].iloc[0] if len(row_meta) > 0 else 'N/A'
            print(f"   - {col}: {sample_val}")
        if len(row_meta.columns) > 15:
            print(f"   ... and {len(row_meta.columns) - 15} more")
        
        # 检查必需的字段
        required_fields = ['pert_id']
        missing_fields = [f for f in required_fields if f not in row_meta.columns]
        if missing_fields:
            raise ValueError(f"Missing required fields in metadata: {missing_fields}")
        
        # 确定细胞系ID字段
        cell_id_col = None
        for possible_col in ['cell_id', 'cell_iname', 'cell_mfc_name']:
            if possible_col in row_meta.columns:
                cell_id_col = possible_col
                print(f"\n✓ Using '{cell_id_col}' as cell line identifier")
                break
        
        if cell_id_col is None:
            print(f"\n⚠️  Warning: Cannot find cell line identifier column")
            print(f"   Available columns: {list(row_meta.columns)}")
        
        # 累积布尔掩码
        valid_mask = np.ones(len(row_meta), dtype=bool)
        
        initial_compounds = row_meta['pert_id'].nunique()
        print(f"\nInitial compounds: {initial_compounds:,}")
        
        # ========== Filter 1: Remove DOS compounds ==========
        if remove_dos:
            print(f"\n{'='*80}")
            print(f"FILTER 1: Remove DOS compounds")
            print(f"{'='*80}")
            
            # 方法1: 通过pert_id识别DOS (BRD-KXXXXXXXXX 通常是DOS)
            # 方法2: 通过cmap_name识别
            dos_mask = ~row_meta['pert_id'].str.contains('DOS|BRD-K', case=False, na=False)
            
            # 如果有pert_type字段（虽然示例中没有，但某些版本可能有）
            if 'pert_type' in row_meta.columns:
                dos_mask &= row_meta['pert_type'].isin(['trt_cp', 'CP'])
            
            valid_mask &= dos_mask
            n_removed = (~dos_mask).sum()
            
            print(f"  Removed {n_removed:,} DOS observations")
            print(f"  Remaining samples: {valid_mask.sum():,}")
            print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
        
        # ========== Filter 2: Minimum observations per compound ==========
        print(f"\n{'='*80}")
        print(f"FILTER 2: Remove compounds with <{min_observations_per_compound} observations")
        print(f"{'='*80}")
        
        valid_meta = row_meta[valid_mask]
        obs_counts = valid_meta.groupby('pert_id').size()
        valid_perts = obs_counts[obs_counts >= min_observations_per_compound].index
        
        print(f"  Compounds with ≥{min_observations_per_compound} observations: "
              f"{len(valid_perts):,}/{obs_counts.nunique():,}")
        
        obs_mask = row_meta['pert_id'].isin(valid_perts)
        valid_mask &= obs_mask
        
        print(f"  Remaining samples: {valid_mask.sum():,}")
        print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
        
        del valid_meta, obs_counts, obs_mask
        gc.collect()
        
        # ========== Filter 3: Cosine similarity ==========
        print(f"\n{'='*80}")
        print(f"FILTER 3: Remove observations with cosine similarity <{min_replicate_similarity}")
        print(f"{'='*80}")
        
        valid_indices = np.where(valid_mask)[0]
        valid_matrix = matrix[valid_mask]
        valid_pert_ids = row_meta.loc[valid_mask, 'pert_id'].reset_index(drop=True)
        
        nearest_similarities = self.calculate_cosine_similarity_to_nearest_replicate(
            valid_matrix, 
            valid_pert_ids
        )
        
        full_similarities = np.zeros(len(row_meta), dtype=np.float32)
        full_similarities[valid_indices] = nearest_similarities
        
        sim_mask = (full_similarities >= min_replicate_similarity) | (~valid_mask)
        n_removed_sim = (~sim_mask & valid_mask).sum()
        valid_mask &= sim_mask
        
        print(f"  Removed {n_removed_sim:,} low-similarity observations")
        print(f"  Remaining samples: {valid_mask.sum():,}")
        
        del valid_matrix, valid_pert_ids, nearest_similarities, full_similarities, sim_mask
        gc.collect()
        
        # ========== Filter 4: Dose selection ==========
        print(f"\n{'='*80}")
        print(f"FILTER 4: Select most frequent dose in range {dose_range[0]}-{dose_range[1]} µM")
        print(f"{'='*80}")
        
        if 'pert_dose' in row_meta.columns:
            # 解析剂量
            row_meta['dose_value'] = pd.to_numeric(
                row_meta['pert_dose'].astype(str).str.extract(r'([\d.]+)')[0], 
                errors='coerce'
            )
            
            row_meta['dose_unit'] = row_meta['pert_dose'].astype(str).str.extract(r'([a-zA-Z]+)')[0]
            
            def convert_to_uM(row):
                if pd.isna(row['dose_value']):
                    return np.nan
                value = row['dose_value']
                unit = str(row['dose_unit']).lower() if pd.notna(row['dose_unit']) else 'um'
                
                if 'nm' in unit:
                    return value / 1000
                elif 'mm' in unit:
                    return value * 1000
                else:
                    return value
            
            row_meta['dose_uM'] = row_meta.apply(convert_to_uM, axis=1)
            
            dose_mask = (
                (row_meta['dose_uM'] >= dose_range[0]) & 
                (row_meta['dose_uM'] <= dose_range[1])
            )
            
            print(f"  Samples in valid dose range: {dose_mask.sum():,}")
            
            # 对每个化合物选择最常见剂量
            dose_final_mask = np.zeros(len(row_meta), dtype=bool)
            valid_meta = row_meta[valid_mask & dose_mask]
            
            for pert_id in valid_meta['pert_id'].unique():
                pert_data = valid_meta[valid_meta['pert_id'] == pert_id]
                dose_counts = pert_data['dose_uM'].value_counts()
                
                if len(dose_counts) > 0:
                    most_common_dose = dose_counts.index[0]
                    dose_final_mask |= (
                        (row_meta['pert_id'] == pert_id) & 
                        (row_meta['dose_uM'] == most_common_dose)
                    )
            
            n_before = valid_mask.sum()
            valid_mask &= dose_final_mask
            n_removed = n_before - valid_mask.sum()
            
            print(f"  Removed {n_removed:,} observations (invalid or non-modal dose)")
            print(f"  Remaining samples: {valid_mask.sum():,}")
            
            del valid_meta, dose_mask, dose_final_mask
            gc.collect()
        else:
            print(f"  ⚠️  Warning: 'pert_dose' column not found, skipping dose filter")
        
        # ========== Filter 5: Timepoint selection ==========
        print(f"\n{'='*80}")
        print(f"FILTER 5: Keep only measurements at {valid_timepoints}")
        print(f"{'='*80}")
        
        if 'pert_time' in row_meta.columns:
            row_meta['time_normalized'] = row_meta['pert_time'].astype(str).str.lower().str.strip()
            
            # 规范化时间点
            normalized_valid_times = set()
            for t in valid_timepoints:
                t_lower = t.lower().strip()
                normalized_valid_times.add(t_lower)
                normalized_valid_times.add(t_lower.replace(' ', ''))
                normalized_valid_times.add(t_lower.replace('h', ' h'))
                normalized_valid_times.add(t_lower.replace('h', 'hr'))
            
            time_mask = row_meta['time_normalized'].isin(normalized_valid_times)
            
            if time_mask.sum() == 0:
                print(f"  ⚠️  Warning: No samples match timepoints {valid_timepoints}")
                print(f"  Available timepoints (top 10):")
                for tp, count in row_meta['pert_time'].value_counts().head(10).items():
                    print(f"    - '{tp}': {count:,} samples")
                print(f"  Skipping timepoint filter...")
            else:
                n_before = valid_mask.sum()
                valid_mask &= time_mask
                n_removed = n_before - valid_mask.sum()
                
                print(f"  Removed {n_removed:,} observations (invalid timepoint)")
                print(f"  Remaining samples: {valid_mask.sum():,}")
            
            del time_mask
            gc.collect()
        else:
            print(f"  ⚠️  Warning: 'pert_time' column not found, skipping timepoint filter")
        
        # ========== Filter 6: Cell line count ==========
        print(f"\n{'='*80}")
        print(f"FILTER 6: Remove compounds in <{min_cell_lines} or >{max_cell_lines} cell lines")
        print(f"{'='*80}")
        
        if cell_id_col is not None:
            valid_meta = row_meta[valid_mask]
            cell_line_counts = valid_meta.groupby('pert_id')[cell_id_col].nunique()
            valid_perts_cell = cell_line_counts[
                (cell_line_counts >= min_cell_lines) & 
                (cell_line_counts <= max_cell_lines)
            ].index
            
            print(f"  Compounds in {min_cell_lines}-{max_cell_lines} cell lines: "
                  f"{len(valid_perts_cell):,}/{len(cell_line_counts):,}")
            
            cell_mask = row_meta['pert_id'].isin(valid_perts_cell)
            n_before = valid_mask.sum()
            valid_mask &= cell_mask
            n_removed = n_before - valid_mask.sum()
            
            print(f"  Removed {n_removed:,} observations")
            print(f"  Remaining samples: {valid_mask.sum():,}")
            print(f"  Remaining compounds: {row_meta.loc[valid_mask, 'pert_id'].nunique():,}")
            
            del valid_meta, cell_line_counts, cell_mask
            gc.collect()
        else:
            print(f"  ⚠️  Warning: Cell line column not found, skipping cell line filter")
        
        # ========== 最终提取数据 ==========
        print(f"\n{'='*80}")
        print(f"✅ FINAL DATASET - Extracting filtered data")
        print(f"{'='*80}")
        
        # 只在最后提取数据
        final_matrix = matrix[valid_mask].copy()
        final_meta = row_meta[valid_mask].reset_index(drop=True)
        
        del matrix
        gc.collect()
        
        print(f"  Extracted {len(final_matrix):,} samples")
        print(f"  Memory usage: {final_matrix.nbytes / (1024**3):.2f} GB")
        
        # 创建标签
        unique_perts = sorted(final_meta['pert_id'].unique())
        pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        labels = np.array([pert_to_idx[p] for p in final_meta['pert_id']], dtype=np.int32)
        
        final_compounds = len(unique_perts)
        final_samples = len(final_matrix)
        
        if cell_id_col:
            final_cells = final_meta[cell_id_col].nunique()
        else:
            final_cells = 'Unknown'
        
        print(f"\n  Total samples: {final_samples:,}")
        print(f"  Total compounds: {final_compounds:,}")
        print(f"  Cell lines: {final_cells}")
        print(f"  Gene features: {final_matrix.shape[1]}")
        
        compound_obs = final_meta.groupby('pert_id').size()
        print(f"  Samples per compound (mean): {final_samples / final_compounds:.1f}")
        print(f"  Samples per compound (median): {compound_obs.median():.0f}")
        
        n_compounds_100plus = (compound_obs > 100).sum()
        print(f"  Compounds with >100 observations: {n_compounds_100plus:,}")
        
        print(f"\n📊 Comparison with paper results:")
        print(f"  Paper: 425,242 obs, 9,597 compounds, 52 cell lines")
        print(f"  Ours:  {final_samples:,} obs, {final_compounds:,} compounds, {final_cells} cell lines")
        
        if initial_compounds > 0:
            retention_rate = (final_compounds / initial_compounds) * 100
            print(f"  Compound retention rate: {retention_rate:.1f}%")
        
        # === 新增：基因名优先使用 geneinfo_beta.txt 中的 landmark gene_symbol ===
        if (self.gene_info is not None) and hasattr(self, "landmark_col_indices"):
            # 先取出标记为 landmark 的行，再按照 landmark_col_indices 的顺序取 gene_symbol
            gi = self.gene_info
            landmark_mask = gi['feature_space'] == 'landmark'
            landmark_geneinfo = gi[landmark_mask]
            
            # landmark_geneinfo 的行顺序与 self.landmark_col_indices 一致
            gene_names = list(landmark_geneinfo['gene_symbol'].values)
            print(f"  Using geneinfo_beta.txt for gene names (landmark gene_symbol)")
        else:
            # 回退方案：还是从 col_meta 里找点能用的
            gene_name_col = None
            for possible_col in ['gene_symbol', 'pr_gene_symbol', 'symbol', 'id', 'gene_id', 'pr_gene_id']:
                if possible_col in col_meta.columns:
                    gene_name_col = possible_col
                    print(f"  Using '{gene_name_col}' for gene names (from col_meta)")
                    break

            if gene_name_col is None:
                gene_name_col = col_meta.columns[0]
                print(f"  Warning: Using '{gene_name_col}' as fallback for gene names (from col_meta)")

            gene_names = list(col_meta[gene_name_col].values)

        training_data = {
            'X': final_matrix,
            'y': labels,
            'folds': np.zeros(len(final_matrix), dtype=np.int32),
            'sample_meta': final_meta,
            'metadata': final_meta,
            'gene_names': gene_names,
            'compound_names': list(unique_perts),
            'pert_to_idx': pert_to_idx
        }

        gc.collect()
        
        return training_data
    
    def create_3fold_splits(self, training_data, random_state=42):
        """创建3折交叉验证划分"""
        np.random.seed(random_state)
        
        sample_meta = training_data['sample_meta']
        
        print(f"\n{'='*80}")
        print(f"🎲 Creating 3-fold cross-validation splits")
        print(f"{'='*80}")
        
        folds = np.zeros(len(sample_meta), dtype=np.int32)
        
        for pert_id in sample_meta['pert_id'].unique():
            pert_mask = sample_meta['pert_id'] == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            np.random.shuffle(pert_indices)
            n_samples = len(pert_indices)
            
            fold_sizes = [n_samples // 3] * 3
            for i in range(n_samples % 3):
                fold_sizes[i] += 1
            
            start_idx = 0
            for fold_id, size in enumerate(fold_sizes):
                end_idx = start_idx + size
                folds[pert_indices[start_idx:end_idx]] = fold_id
                start_idx = end_idx
        
        training_data['folds'] = folds
        
        print(f"\n📊 Fold statistics:")
        for fold_id in range(3):
            fold_mask = folds == fold_id
            n_samples = fold_mask.sum()
            n_compounds = sample_meta[fold_mask]['pert_id'].nunique()
            print(f"   Fold {fold_id}: {n_samples:,} samples, {n_compounds:,} compounds")
        
        return training_data


def main():
    """主程序 - 内存优化版（修正版）"""
    print("=" * 80)
    print("🧬 DRUGREFLECTOR DATA PREPROCESSING (Memory-Optimized v2)")
    print("   Updated for actual LINCS 2020 data format")
    print("=" * 80)
    
    data_dir = "E:/科研/Models/drugreflector/datasets/LINCS2020"
    loader = LINCS2020DataLoader(data_dir)
    
    try:
        # Step 1: 加载元数据
        print("\n" + "=" * 80)
        print("STEP 1: Loading metadata")
        print("=" * 80)
        gene_info = loader.load_gene_info()
        cell_info = loader.load_cell_info()
        compound_info = loader.load_compound_info()
        
        # Step 2: 加载Level 4 signatures
        print("\n" + "=" * 80)
        print("STEP 2: Loading Level 4 signatures")
        print("=" * 80)
        matrix, row_meta, col_meta = loader.load_level4_signatures()
        
        # Step 3: 准备训练数据
        print("\n" + "=" * 80)
        print("STEP 3: Preparing training data")
        print("=" * 80)
        training_data = loader.prepare_training_data(
            min_observations_per_compound=5,
            min_replicate_similarity=0.12,
            dose_range=(1.0, 20.0),
            valid_timepoints=['6 h', '24 h', '6h', '24h', '6hr', '24hr', '6 hr', '24 hr'],
            min_cell_lines=5,
            max_cell_lines=40,
            remove_dos=True
        )
        
        # Step 4: 创建3折划分
        print("\n" + "=" * 80)
        print("STEP 4: Creating 3-fold splits")
        print("=" * 80)
        training_data = loader.create_3fold_splits(training_data)
        
        # Step 5: 保存
        print("\n" + "=" * 80)
        print("STEP 5: Saving processed data")
        print("=" * 80)
        output_dir = Path("E:/科研/Models/drugreflector/processed_data")
        output_dir.mkdir(exist_ok=True, parents=True)
        
        output_file = output_dir / "training_data_lincs2020_paper_compliant.pkl"
        print(f"💾 Saving to: {output_file}")
        
        with open(output_file, 'wb') as f:
            pickle.dump(training_data, f, protocol=4)
        
        print(f"✓ Saved successfully!")
        print(f"   File size: {output_file.stat().st_size / (1024**2):.1f} MB")
        
        # 最终摘要
        print("\n" + "=" * 80)
        print("✅ DATA PREPARATION COMPLETE!")
        print("=" * 80)
        print(f"📁 Output file: {output_file}")
        print(f"\n📊 Final dataset summary:")
        print(f"   • Total samples: {len(training_data['X']):,}")
        print(f"   • Total compounds: {len(training_data['compound_names']):,}")
        print(f"   • Gene features: {training_data['X'].shape[1]}")
        print(f"   • Memory usage: {training_data['X'].nbytes / (1024**3):.2f} GB")
        print(f"\n🎯 Ready for training!")
        
        return training_data
        
    except Exception as e:
        print(f"\n" + "=" * 80)
        print("❌ ERROR DURING DATA PREPARATION")
        print("=" * 80)
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        
        import traceback
        traceback.print_exc()
        
        print(f"\n💡 Troubleshooting:")
        print(f"   1. Verify all files are in: {Path(data_dir).absolute()}")
        print(f"   2. Check file formats match the expected TSV structure")
        print(f"   3. Ensure sufficient RAM (32GB+ recommended)")
        
        return None


if __name__ == "__main__":
    training_data = main()

🧬 DRUGREFLECTOR DATA PREPROCESSING (Memory-Optimized v2)
   Updated for actual LINCS 2020 data format

🔬 LINCS 2020 Data Loader Initialized (Memory-Optimized v2)
Data directory: E:\科研\Models\drugreflector\datasets\LINCS2020
Dataset: Expanded CMap LINCS Resource 2020


STEP 1: Loading metadata
📖 Loading gene information...
   File: geneinfo_beta.txt
   ✓ Loaded 12,328 genes
   ✓ Columns: ['gene_id', 'gene_symbol', 'ensembl_id', 'gene_title', 'gene_type', 'src', 'feature_space']
   ✓ Using 'feature_space' column to identify landmarks
   ✓ Landmark genes: 978
   ✓ Expected: 978

   Landmark column indices (first 10): [2154 2155 2156 2157 2158 2159 2160 2161 2162 2163]
   Sample IDs for matching: ['54807', '51203', '9709', '332', '56889']
   Sample symbols for matching: ['POLR1C', 'ATF6', 'PPIE', 'KLHL9', 'MAST2']

📖 Loading cell information...
   File: cellinfo_beta.txt
   ✓ Loaded 240 cell lines
   ✓ Columns: ['cell_iname', 'cellosaurus_id', 'donor_age', 'donor_age_death', 'donor_disease

Traceback (most recent call last):
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_8724\3830637168.py", line 944, in main
    matrix, row_meta, col_meta = loader.load_level4_signatures()
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_8724\3830637168.py", line 441, in load_level4_signatures
    self.load_instance_info()
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_8724\3830637168.py", line 210, in load_instance_info
    raise ValueError(
ValueError: 'inst_id' column not found in instinfo_beta.txt. Available columns: ['bead_batch', 'nearest_dose', 'pert_dose', 'pert_dose_unit', 'pert_idose', 'pert_time', 'pert_itime', 'pert_time_unit', 'cell_mfc_name', 'pert_mfc_id', 'det_plate', 'det_well', 'rna_plate', 'rna_well', 'count_mean', 'count_cv', 'qc_f_logp', 'qc_iqr', 'qc_slope', 'pert_id', 'sample_id', 'pert_type', 'cell_iname', 'qc_pass', 'dyn_range', 'inv_level_10', 'build_name', 'failure_mode', 'project_code

🧬 DRUGREFLECTOR DATA PREPROCESSING (Memory-Optimized)

🔬 LINCS 2020 Data Loader (Memory-Optimized)
Data directory: E:\科研\Models\drugreflector\datasets\LINCS2020
Max memory: 16 GB
Chunk size: 50,000 samples

📖 Loading gene information...
   ✓ Landmark genes: 978
📖 Loaded 240 cell lines
📖 Loaded 39,321 compounds

📖 Reading GCTX metadata: level4_beta_trt_cp_n1805898x12328.gctx
✓ Metadata loaded:
  Samples: 12,328
  Genes: 1805898
  Matrix shape: (1805898, 12328)

❌ Error: LINCS2020DataLoaderOptimized.filter_metadata_first() got an unexpected keyword argument 'min_replicate_similarity'


Traceback (most recent call last):
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_25884\2025916354.py", line 498, in main
    training_data = loader.prepare_training_data(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_25884\2025916354.py", line 390, in prepare_training_data
    filtered_meta = self.filter_metadata_first(sample_meta, **filter_params)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: LINCS2020DataLoaderOptimized.filter_metadata_first() got an unexpected keyword argument 'min_replicate_similarity'
